# Ensemble Methods Applied

Agenda:
- Review code for Voting Classifier, Bagging Classifier, and Random Forest


## Import and Prep Titanic dataset

In [56]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier


In [57]:
# Read in data and split data to be used in the models
titanic = pd.read_csv('https://raw.githubusercontent.com/learn-co-students/nyc-ds-033020-lectures/master/Mod_3/decision_trees/cleaned_titanic.csv', index_col='PassengerId')



In [58]:
# Create matrix of features
X = titanic.drop('Survived', axis = 1) # grabs everything else but 'Survived'

# Create target variable
y = titanic['Survived'] # y is the column we're trying to predict

# Create a list of the features being used in the 
feature_cols = X.columns

In [59]:
# Use x and y variables to split the training data into train and test set then scale that data

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

scaler = StandardScaler()  
scaler.fit(X_train)

X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)

## Fit a KNN model

In [5]:
from sklearn.neighbors import KNeighborsClassifier

In [6]:
knn = KNeighborsClassifier(n_neighbors=9)

In [7]:
knn.fit(X_train, y_train)

knn_preds = knn.predict(X_test)

knn_f1 = metrics.f1_score(y_test, knn_preds)


print(knn_f1)

0.7975460122699386


## Fit a Logistic Regression model 

In [8]:
from sklearn.linear_model import LogisticRegression

In [9]:
lr = LogisticRegression(class_weight='balanced')

In [10]:
lr.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

In [11]:
lr_preds = lr.predict(X_test)

lr_f1 = metrics.f1_score(y_test, lr_preds)

print(lr_f1)

0.8066298342541436


## Fit a Decision Tree Classifier

In [12]:
from sklearn.tree import DecisionTreeClassifier

In [13]:
dtc = DecisionTreeClassifier(max_depth=5, class_weight='balanced')

dtc.fit(X_train, y_train)

dtc_preds  = dtc.predict(X_test)

dtc_f1 = metrics.f1_score(y_test, dtc_preds)

print(dtc_f1)

0.8047337278106509


## Combine three models using Voting Classifier

In [14]:
from sklearn.ensemble import VotingClassifier


For the estimators, we must provide a list of tuples. The first value in the tuple is is the name given to the model/estimator in the second value. SKlearn requires this because there is additional functionality where you can access information about the specific models, so you need to name the models to access them later.  

In [15]:
voting_clf = VotingClassifier(
                estimators=[('logreg', lr), ('knneighbors', knn), ('decisiontree', dtc)], 
                voting='hard')

voting_clf.fit(X_train, y_train)

vc_preds = voting_clf.predict(X_test)

vc_f1 = metrics.f1_score(y_test, vc_preds)

print(vc_f1)

0.8160919540229884


### Use a voting classifier with multiple Logistic regression models 

In [16]:

C_param_range = [0.001,0.01,0.1,1,10]
titles = ['lr_0_001', 'lr_0_01', 'lr_0_1', 'lr_1', 'lr_10']

params = dict(zip(titles, C_param_range)) 
models = {}

table = pd.DataFrame(columns = ['C_parameter','F1'])
table['C_parameter'] = C_param_range
j = 0

for k , v  in params.items():
    
    # Create model using different value for c  
    lr = LogisticRegression(penalty = 'l2', C = v, random_state = 1, class_weight='balanced')
    
    #save the model to a dictionary to use later in our voting classifiers
    models[k]= lr
    
    #the steps below this point are unnecessary in order to create a voting classifier, 
    #but it is easy to fit the model and see how performance changes for different levels of regularization
    lr.fit(X_train, y_train)
    
    # Predict using model
    y_preds = lr.predict(X_test)

    # Saving accuracy score in table
    table.iloc[j,1] = metrics.f1_score(y_test, y_preds)
    j += 1



In [17]:
#review performance for different levels of C
table


,C_parameter,F1
0,0.001,0.735135
1,0.010,0.751381
2,0.100,0.804469
3,1.000,0.80663
4,10.000,0.80663


In [18]:
#invesitgate the models D=dictionary
list(models.items())

[('lr_0_001',
  LogisticRegression(C=0.001, class_weight='balanced', random_state=1)),
 ('lr_0_01',
  LogisticRegression(C=0.01, class_weight='balanced', random_state=1)),
 ('lr_0_1',
  LogisticRegression(C=0.1, class_weight='balanced', random_state=1)),
 ('lr_1', LogisticRegression(C=1, class_weight='balanced', random_state=1)),
 ('lr_10', LogisticRegression(C=10, class_weight='balanced', random_state=1))]

Now that we have programmatically created multiple logistic regression models, let's use them in an ensemble model

In [19]:
lr_voting = VotingClassifier(estimators=list(models.items()), 
                              voting='hard')

lr_voting.fit(X_train, y_train)

lrv_preds = lr_voting.predict(X_test)

lrv_f1 = metrics.f1_score(y_test, lrv_preds)

print(lrv_f1)

0.8044692737430168


## Fit a Bagging Classifier for a Logistic Regression model. 

In [20]:
X_train.shape

(666, 9)

In [21]:
bc_lr = BaggingClassifier(
            base_estimator=LogisticRegression(random_state = 1, class_weight='balanced'), 
            n_estimators= 100,
            max_samples= 0.8,
            max_features= 6,
            oob_score= True
                )

In [22]:
bc_lr.fit(X_train, y_train)



BaggingClassifier(base_estimator=LogisticRegression(class_weight='balanced',
                                                    random_state=1),
                  max_features=6, max_samples=0.8, n_estimators=100,
                  oob_score=True)

In [23]:
# Use the oob_score to get some idea of how the model performs on a validation set

bc_lr.oob_score_

0.7777777777777778

In [24]:
# See how the model performs on the test set

bc_lr_preds = bc_lr.predict(X_test)

bc_lr_f1 = metrics.f1_score(y_test, bc_lr_preds)

print(bc_lr_f1)

0.7777777777777778


***What is the difference in the `VotingClassifier` algorithm and the `BaggingClassifier` algorithm?***

Your answer:

**What is the difference between a BaggingClassifier that uses a decision tree as the base estimator and a Random Forest Classifier?**

A random forest classifier will take a sample of features at each node, where as a bagging classifier will take a sample of features at to use for the whole model. 

# Fitting a Random Forest Classifier

In [25]:
# Instantiate the classifier using 100 trees
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state = 1, n_estimators=100, max_depth=2, max_features=4)

In [26]:
#let's look at all the different default features
rfc

RandomForestClassifier(max_depth=2, max_features=4, random_state=1)

In [27]:
#fit the model to the training data
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, max_features=4, random_state=1)

In [28]:
#use the fitted model to predict on the test data
rfc_preds = rfc.predict(X_test)

rfc_f1 = metrics.f1_score(y_test, rfc_preds)

# checking accuracy on the test data
print('Test F1 score: ', rfc_f1)

Test F1 score:  0.7074829931972789


***Change the parameters and see whether you can improve the performance of the model***

### GridsearchCV with Random Forest

Let's use grid search to identify the best tuning parameters to use for a random forest model. 

In [29]:
from sklearn.model_selection import GridSearchCV

In [30]:
RandomForestClassifier()

RandomForestClassifier()

In [31]:
#create a dictionary of all the parameters you want to tune
param_grid = { 
    'n_estimators': [100,300,500,700,1000],
    'criterion': ['gini', 'entropy'],
    'max_depth': list(range(2,10)),
    'max_features': list(range(3,7))
}

In [32]:
#create a grid search object and fit it to the data

grid_tree=GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='f1', verbose=1, n_jobs=-1)

In [33]:
grid_tree.fit(X_train, y_train)

Fitting 5 folds for each of 320 candidates, totalling 1600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   35.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 1600 out of 1600 | elapsed:  4.8min finished


GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8, 9],
                         'max_features': [3, 4, 5, 6],
                         'n_estimators': [100, 300, 500, 700, 1000]},
             scoring='f1', verbose=1)

In [34]:
### Identify the best params 



# Single best score achieved across all params (min_samples_split)
print(grid_tree.best_score_)

# Dictionary containing the parameters (min_samples_split) used to generate that score
print(grid_tree.best_params_)

# Actual model object fit with those best parameters
# Shows default parameters that we did not specify
print(grid_tree.best_estimator_)
#Identify the best score during fitting with cross-validation


0.756228569821669
{'criterion': 'gini', 'max_depth': 8, 'max_features': 4, 'n_estimators': 700}
RandomForestClassifier(max_depth=8, max_features=4, n_estimators=700)


In [35]:
#Predict the response for test dataset
y_pred = grid_tree.best_estimator_.predict(X_test)

# Model F1, how often is the classifier correct?
print("F1:",metrics.f1_score(y_test, y_pred))

F1: 0.8125


In [53]:
import pandas as pd
import numpy as np

df1 = pd.DataFrame(np.random.randn(5, 4), columns= ['A', 'B', 'C', 'D'])
df2 = pd.DataFrame(np.random.randn(4, 2), columns= ['A' , 'Y'])
df3 = pd.DataFrame(np.random.randn(6, 3), columns= ['A', 'N', 'O'])
df4 = pd.DataFrame(np.random.randn(8, 3), columns= ['A', 'K', 'L'])

In [54]:
df3

,A,N,O
0,1.053874,-0.870946,-0.943606
1,-0.653308,0.098896,0.224423
2,-0.682303,-0.737286,0.425462
3,-1.859590,1.416239,2.177101
4,1.482216,0.094321,-0.164909
5,0.537344,-0.015710,0.502843


In [55]:
pd.concat([df1,df2,df3,df4])

,A,B,C,D,Y,N,O,K,L
0,0.418008,0.017556,-0.235417,-0.771987,NaN,NaN,NaN,NaN,NaN
1,0.963560,1.699745,-1.950171,2.012574,NaN,NaN,NaN,NaN,NaN
2,-0.719708,-0.069473,2.181455,0.160253,NaN,NaN,NaN,NaN,NaN
3,-1.017701,0.876463,-0.564596,0.487549,NaN,NaN,NaN,NaN,NaN
4,-0.788705,0.495291,0.597326,-0.603854,NaN,NaN,NaN,NaN,NaN
0,1.034856,NaN,NaN,NaN,-1.109159,NaN,NaN,NaN,NaN
1,-0.287476,NaN,NaN,NaN,0.389333,NaN,NaN,NaN,NaN
2,0.645574,NaN,NaN,NaN,0.093363,NaN,NaN,NaN,NaN
3,0.394108,NaN,NaN,NaN,0.739211,NaN,NaN,NaN,NaN
0,1.053874,NaN,NaN,NaN,NaN,-0.870946,-0.943606,NaN,NaN


In [ ]:
df